# Creacion de los Tf Records

In [ ]:
# Funciones auxiliares

def serialize_example(audio, label):
  feature = {
      'audio': tf.train.Feature(bytes_list=tf.train.BytesList(value=[audio.numpy()])),
      'label': tf.train.Feature(int64_list=tf.train.Int64List(value=label.numpy()))
  }
  example = tf.train.Example(features=tf.train.Features(feature=feature))
  return example.SerializeToString()

def save_to_tfrecord(df, audio_folder: Path, label_to_index, num_classes, output_path):
  with tf.io.TFRecordWriter(str(output_path)) as writer:
    for _, row in df.iterrows():
      wav_path = audio_folder / row['fname']
      if not wav_path.exists():
        continue
      audio_binary = tf.io.read_file(str(wav_path))
      label = labels_to_multihot(row['labels'], label_to_index, num_classes)
      # Convert the NumPy array to a TensorFlow tensor
      label_tensor = tf.constant(label, dtype=tf.int64)
      example = serialize_example(audio_binary, label_tensor)
      writer.write(example)

def serialize_example_test(audio):
  feature = {
      'audio': tf.train.Feature(bytes_list=tf.train.BytesList(value=[audio.numpy()]))
  }
  example = tf.train.Example(features=tf.train.Features(feature=feature))
  return example.SerializeToString()

def save_to_tfrecord_test(df, audio_folder: Path, output_path):
  with tf.io.TFRecordWriter(str(output_path)) as writer:
    for _, row in df.iterrows():
      wav_path = audio_folder / row['fname']
      if not wav_path.exists():
        continue
      audio_binary = tf.io.read_file(str(wav_path))
      # Convert the NumPy array to a TensorFlow tensor
      example = serialize_example_test(audio_binary)
      writer.write(example)

In [ ]:
# Utilizar un codigo similar para la creacion de los Tf Records

save_to_tfrecord(train_curated_df, base_dir / 'train_curated', label_to_index, len(all_labels), base_dir / 'train_curated.tfrecord')
save_to_tfrecord_test(test_df, base_dir / 'test_resampled/test_resampled', base_dir / 'test_resampled.tfrecord')

